# Add MW capacity and basic post-processing

In [1]:
import os, json, geojson, sys
from shapely import geometry
from area import area

import geopandas as gpd
import georasters as gr
import numpy as np
import itertools

/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/geopandas/_compat.py:78: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### constants for MW conversion

In [ ]:
GCR = 0.35 # ground coverage ratio
ILR = 1.1 # inverter loading ratio
EFF = 0.15 # panel efficiency

### IO

In [3]:
root = '/home/lucas/repos/solar-pv-global-inventory'

In [5]:
gdf = gpd.read_file(os.path.join(root,'data','ABCD_lccapacity.geojson'))

In [6]:
gdf.columns

Index(['area', 'confidence', 'install_date', 'install_date_ints', 'SPOT_ids_0',
       'S2_ids_0', 'iso-3166-1', 'iso-3166-2', 'land_cover_CORINE_2006',
       'land_cover_CORINE_2012', 'land_cover_CORINE_2018',
       'land_cover_MODIS_2006', 'land_cover_MODIS_2007',
       'land_cover_MODIS_2008', 'land_cover_MODIS_2009',
       'land_cover_MODIS_2010', 'land_cover_MODIS_2011',
       'land_cover_MODIS_2012', 'land_cover_MODIS_2013',
       'land_cover_MODIS_2014', 'land_cover_MODIS_2015',
       'land_cover_MODIS_2016', 'land_cover_MODIS_2017',
       'land_cover_MODIS_2018', 'SPOT_ids_1', 'SPOT_ids_2', 'S2_ids_1',
       'SPOT_ids_3', 'SPOT_ids_4', 'SPOT_ids_5', 'SPOT_ids_6', 'SPOT_ids_7',
       'SPOT_ids_8', 'land_cover_CDL_2006', 'land_cover_CDL_2007',
       'land_cover_CDL_2008', 'land_cover_CDL_2009', 'land_cover_CDL_2010',
       'land_cover_CDL_2011', 'land_cover_CDL_2012', 'land_cover_CDL_2013',
       'land_cover_CDL_2014', 'land_cover_CDL_2015', 'land_cover_CDL_2016',
  

In [ ]:
rast_pvout = gr.load_tiff(os.path.join(root,'data','irradiance_maps','PVOUT.tif'))

In [ ]:
rast_gti = gr.load_tiff(os.path.join(root,'data','irradiance_maps','GTI.tif'))

In [ ]:
NDV, xsize, ysize, GeoT, Projection, DataType = gr.get_geo_info(os.path.join(root,'data','irradiance_maps','PVOUT.tif'))
meta_pvout = {'NVD':NDV, 'xsize':xsize,'ysize':ysize,'GeoT':GeoT,'Projection':Projection,'DataType':DataType}

In [ ]:
NDV, xsize, ysize, GeoT, Projection, DataType = gr.get_geo_info(os.path.join(root,'data','irradiance_maps','GTI.tif'))
meta_gti = {'NVD':NDV, 'xsize':xsize,'ysize':ysize,'GeoT':GeoT,'Projection':Projection,'DataType':DataType}

### Remap Area

In [ ]:
gdf['area'] = gdf.apply(lambda row: area(geometry.mapping(row['geometry'])), axis=1)

### Drop Tinies

In [ ]:
gdf = gdf[gdf['area']>1]

### map GTI and PVOUT

**GTI** -> solar irradiance in kWh/m^2

**PVOUT** -> unit kWh production in kWh/kWp_dc

In [ ]:
def _map_gti(el):
    pt = el.geometry.representative_point()
    x = pt.x
    y = pt.y
    col, row = gr.map_pixel(x,y,meta_gti['GeoT'][1],meta_gti['GeoT'][-1], meta_gti['GeoT'][0],meta_gti['GeoT'][3])
    val = rast_gti[col,row]
    if not val==np.nan:
        return val
    else:
        ii=1
        while val==np.nan:
            print (ii),
            new_inds = list(itertools.product([col-1,col,col+1],[row-1,row,row+1]))
            vals = [rast_gti[c,r] for c,r in new_inds]
            val = np.nanmean(vals)
            ii+=1
        
        return val

In [ ]:
def _map_pvout(el):
    pt = el.geometry.representative_point()
    x = pt.x
    y = pt.y
    col, row = gr.map_pixel(x,y,meta_gti['GeoT'][1],meta_gti['GeoT'][-1], meta_gti['GeoT'][0],meta_gti['GeoT'][3])
    val = rast_pvout[col,row]
    if not val==np.nan:
        return val
    else:
        ii=1
        while val==np.nan:
            print (ii),
            new_inds = list(itertools.product([col-1,col,col+1],[row-1,row,row+1]))
            vals = [rast_pvout[c,r] for c,r in new_inds]
            val = np.nanmean(vals)
            ii+=1
        
        return val

In [ ]:
gdf['gti'] = gdf.apply(lambda el: _map_gti(el), axis=1)

In [ ]:
gdf['gti'] = gdf['gti'] * 365.25

In [ ]:
gdf['pvout'] = gdf.apply(lambda el: _map_pvout(el), axis=1)

In [ ]:
gdf['pvout'] = gdf['pvout'] * 365.25

In [ ]:
gdf['capacity_mw'] = gdf['area'] * gdf['gti'] * GCR / gdf['pvout'] / ILR * EFF / 1000

In [ ]:
for test_GCR in [0.3,0.35,0.4]:
    
    test_MW = gdf['area'] * gdf['gti'] * test_GCR / gdf['pvout'] / ILR * EFF / 1000
    print (test_GCR, test_MW.sum(), test_MW.sum()/gdf['capacity_mw'].sum())
for test_ILR in [1.05,1.1,1.15]:
    test_MW = gdf['area'] * gdf['gti'] * GCR / gdf['pvout'] / test_ILR * EFF / 1000
    print (test_ILR, test_MW.sum(), test_MW.sum()/gdf['capacity_mw'].sum())
for test_EFF in [0.12,0.15,0.18]:
    test_MW = gdf['area'] * gdf['gti'] * GCR / gdf['pvout'] / ILR * test_EFF / 1000
    print (test_EFF, test_MW.sum(), test_MW.sum()/gdf['capacity_mw'].sum())

In [ ]:
1.142*1.048*1.2

In [ ]:
0.857*0.957*0.8

In [ ]:
-> +- 44%

In [ ]:
gdf['capacity_mw'] = gdf['area'] * gdf['gti'] * GCR / gdf['pvout'] / ILR * EFF / 1000 # m^2 * kwh/m^2 * GCR / (kwh/kwp) / ILR
gdf['capacity_mw_big'] = gdf['area'] * gdf['gti'] * .40 / gdf['pvout'] / 1.05 * .18 / 1000 # m^2 * kwh/m^2 * GCR / (kwh/kwp) / ILR
gdf['capacity_mw_small'] = gdf['area'] * gdf['gti'] * .30 / gdf['pvout'] / 1.15 * .12 / 1000 # m^2 * kwh/m^2 * GCR / (kwh/kwp) / ILR

In [ ]:
gdf['big_capacity_mw'] = gdf['area'] * gdf['gti'] * 0.5 / gdf['pvout'] / 1.05 * .18 / 1000 # m^2 * kwh/m^2 * GCR / (kwh/kwp) / ILR

In [ ]:
gdf['dummy_mw'] = gdf['area']*44.2 /1000 /1000

In [ ]:
gdf[['iso-3166-1','capacity_mw', 'capacity_mw_big','capacity_mw_small']].groupby('iso-3166-1').sum().sort_values('capacity_mw', ascending=False).head(21)

In [ ]:
gdf['capacity_mw'].sum()

In [ ]:
gdf['dummy_mw'].sum()

In [ ]:
for iso2 in ['CN','JP','US','DE','IT']:
    print (iso2,gdf[gdf['iso-3166-1']==iso2].capacity_mw.sum())

In [ ]:
for iso2 in ['CN','JP','US','DE','IT']:
    print (iso2,gdf[gdf['iso-3166-1']==iso2].big_capacity_mw.sum())

In [ ]:
for iso2 in ['CN','JP','US','DE','IT']:
    print (iso2,gdf[gdf['iso-3166-1']==iso2].dummy_mw.sum())

In [ ]:
125.0 / 173.5, 12.4/ 46.5, 40.0/40.2, 11.7/38.7, 8.7/15.9

### Example

In [ ]:
for ii in range(20):
    row=ii//2
    col = int(2*(ii%2)) #+ ii%2
    print (row, col)
    print (row, col+1)

In [ ]:
mean_pvout = rast_pvout[rast_pvout>0].mean()

In [ ]:
mean_pvout

In [ ]:
mean_gti = rast_gti[rast_gti>0].mean()

In [ ]:
mean_gti

In [ ]:
10000 * mean_gti*365.25 * GCR / mean_pvout/365.25 / ILR * EFF 

In [ ]:
10000*44.2/1000

In [ ]:
45.2-6.5

### IO

In [ ]:
gdf = gdf.drop(columns=['dummy_mw','big_capacity_mw'])

In [ ]:
gdf.to_file(os.path.join(root,'data','ABCD_lccapacity.geojson'), driver='GeoJSON')